# Exploratory data analysis on chips, by segments and packets and family size

In [16]:
import pandas as pd
merged_data = pd.read_csv("../data/MergedData.csv")
# Overall sales summary


# Total sales
total_sales = merged_data['TOT_SALES'].sum()

# Total transactions (unique TXN_ID)
total_transactions = merged_data['TXN_ID'].nunique()

# Total customers (unique LYLTY_CARD_NBR)
total_customers = merged_data['LYLTY_CARD_NBR'].nunique()

# Average spend per transaction
avg_spend_transaction = merged_data.groupby('TXN_ID')['TOT_SALES'].sum().mean()

# Average spend per customer
avg_spend_customer = merged_data.groupby('LYLTY_CARD_NBR')['TOT_SALES'].sum().mean()

# Average quantity per transaction (basket size)
avg_basket_size = merged_data.groupby('TXN_ID')['PROD_QTY'].sum().mean()

summary = {
    "Total Sales ($)": total_sales,
    "Total Transactions": total_transactions,
    "Total Customers": total_customers,
    "Average Spend per Transaction ($)": avg_spend_transaction,
    "Average Spend per Customer ($)": avg_spend_customer,
    "Average Basket Size (packets)": avg_basket_size
}
summary

{'Total Sales ($)': 511588.2999999999,
 'Total Transactions': 74438,
 'Total Customers': 43625,
 'Average Spend per Transaction ($)': 6.872676589913753,
 'Average Spend per Customer ($)': 11.726952435530087,
 'Average Basket Size (packets)': 1.9076412584970042}

In [17]:
# ---- Product Analysis ----

# Extract packet size in grams from PROD_NAME (look for numbers ending with 'g')
import re

def get_packet_size(name):
    match = re.search(r'(\d+)\s*g', name)
    return int(match.group(1)) if match else None

merged_data['PACK_SIZE'] = merged_data['PROD_NAME'].apply(get_packet_size)

# Extract brand name (take first word of PROD_NAME as brand)
merged_data['BRAND'] = merged_data['PROD_NAME'].str.split().str[0]

# Sales by brand
sales_by_brand = merged_data.groupby('BRAND')['TOT_SALES'].sum().sort_values(ascending=False)

# Sales by packet size
sales_by_packsize = merged_data.groupby('PACK_SIZE')['TOT_SALES'].sum().sort_values(ascending=False)

# Top 10 brands and packet sizes
top_brands = sales_by_brand.head(10)
top_pack_sizes = sales_by_packsize.head(10)
# Variables are all in dollars
top_brands, top_pack_sizes


(BRAND
 Smiths      129237.8
 Doritos     127597.3
 Thins        88852.5
 Cobs         70569.8
 Natural      34272.0
 WW           26655.1
 Tostitos     26474.8
 French        7929.0
 Name: TOT_SALES, dtype: float64,
 PACK_SIZE
 175    183172.3
 170    112396.4
 110     70569.8
 150     47622.9
 380     36367.6
 330     34804.2
 200     16007.5
 160     10647.6
 Name: TOT_SALES, dtype: float64)

In [18]:
# ---- Combined Product + Customer Insights ----

# Average pack size purchased by each segment (Lifestage + Premium)
avg_packsize_by_segment = merged_data.groupby(['LIFESTAGE','PREMIUM_CUSTOMER'])['PACK_SIZE'].mean().sort_values(ascending=False)

# Top brands by segment (which brand dominates in each segment)
brand_sales_by_segment = merged_data.groupby(['LIFESTAGE','PREMIUM_CUSTOMER','BRAND'])['TOT_SALES'].sum().reset_index()

# For each segment, find the top brand
top_brand_per_segment = brand_sales_by_segment.loc[brand_sales_by_segment.groupby(['LIFESTAGE','PREMIUM_CUSTOMER'])['TOT_SALES'].idxmax()]

display(avg_packsize_by_segment.head(10))
display(top_brand_per_segment.head(10))


LIFESTAGE               PREMIUM_CUSTOMER
YOUNG SINGLES/COUPLES   Mainstream          182.293107
MIDAGE SINGLES/COUPLES  Mainstream          182.089709
NEW FAMILIES            Premium             180.556793
OLDER FAMILIES          Budget              180.528368
RETIREES                Premium             180.340146
YOUNG FAMILIES          Budget              180.231225
RETIREES                Budget              179.966318
OLDER SINGLES/COUPLES   Budget              179.925561
OLDER FAMILIES          Premium             179.808368
OLDER SINGLES/COUPLES   Premium             179.783702
Name: PACK_SIZE, dtype: float64

,LIFESTAGE,PREMIUM_CUSTOMER,BRAND,TOT_SALES
1,MIDAGE SINGLES/COUPLES,Budget,Doritos,2563.8
12,MIDAGE SINGLES/COUPLES,Mainstream,Smiths,6302.5
20,MIDAGE SINGLES/COUPLES,Premium,Smiths,3792.4
25,NEW FAMILIES,Budget,Doritos,1468.1
33,NEW FAMILIES,Mainstream,Doritos,1223.8
41,NEW FAMILIES,Premium,Doritos,854.1
52,OLDER FAMILIES,Budget,Smiths,12310.4
60,OLDER FAMILIES,Mainstream,Smiths,7490.1
68,OLDER FAMILIES,Premium,Smiths,5741.4
73,OLDER SINGLES/COUPLES,Budget,Doritos,9120.2
